MLP Trained on Clustercad dataset w/ modified annotations based on mal/mmal presence. 
MAL -> remove configuration (1/2)
MMAL -> no change

see annotation_relable.ipynb for implmenetation.

In [211]:
import pandas as pd
import numpy as np 
from torch.utils.data import TensorDataset, DataLoader
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.optim as optim

In [212]:
import torch
import random
import numpy as np

# Set a fixed seed
seed = 1
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

# Ensure deterministic behavior
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False


In [213]:
ccad_filtered = pd.read_csv("relabled_ccad.csv")
ccad_filtered_embeddings = torch.load("relabled_ccad.pt")
ccad_filtered_embeddings = ccad_filtered_embeddings["embeddings"]

In [214]:
ccad_filtered = ccad_filtered.dropna()

In [215]:
print(len(ccad_filtered))
print(len(ccad_filtered_embeddings))

1157
1167


In [216]:
'''
to_add = ["C","C1", "C2", "B1", "B2", "A2"]
ccad_filtered = ccad_filtered[ccad_filtered["kr_type_annotation"].isin(to_add)]
'''

to_add = ["C","C2","C1","B", "B2","A","A1", "A2","B1"]
ccad_filtered = ccad_filtered[ccad_filtered["kr_type_annotation"].isin(to_add)]

In [217]:
ccad_filtered["new_kr"].unique()

array(['B1', 'C1', 'B', 'A', 'A2', 'A1', 'C', 'B2', 'C2'], dtype=object)

In [218]:
ccad_filtered[ccad_filtered["new_kr"] == "A"]

,Unnamed: 0,index,genbank_accession,mibig_accession,kr_type_annotation,kr_sequence,subunit_name,module_number,length_aa_residues,substrate_annotation,intermediate_smiles,name_in_fasta,protein_sequence,dna_sequence,substrate_type,new_kr,new_st
3,3,3,KX458106.1,BGC0001690.1,A1,GTVLITGGTGALGAEAARWLARSGAEHLLLTSRRGPEAPGAAELAA...,SgnS2,7,1522,"substrate mal, non-loading",CC(=O)C[C@@H](O)CC=[C]C=[C]C=[C]C=[C][C@H](O)C...,BGC0001690.1_SgnS2_mod7,IVIVGMSCRYPGGVASPEDLWRLVSEGTDAISGLPTDRGWDLDALY...,ATCGTCATCGTCGGCATGAGCTGCCGCTACCCGGGCGGCGTCGCCT...,mal,A,mal
7,7,7,KX263301.1,BGC0001670.1,A1,GTTLITGGTGGIGAVLARWLARAGAPHLLLTSRRGPDAPGAQELAA...,monAVIII,11,1525,"substrate mal, non-loading",CCC([CH][CH]C=C(C)[CH][CH]C(=O)C[C@H](O)[C@@H]...,BGC0001670.1_monAVIII_mod11,VAIVGMACRYPGGARSAEDLWELVRDGGDAVAGFPDDRGWDLESLY...,CTGTCCGCTCGCGCCGTCCCTGCCGAGGTCCCGGTCGATGAACGCG...,mal,A,mal
34,34,35,FM173265.1,BGC0000087.1,A2,GTVLVTGGTGGVGGRVARWLATSGAEHLLLVSRSGPEAPGAAELEA...,lasAII,5,1515,"substrate emal, non-loading",CC=C([CH][CH]C=C([CH][C@H](C)[C@H](O)[C@H](CC)...,BGC0000087.1_lasAII_mod5,AIVSMACRFPGGIESPDDLWQYVVQGRDAVSDFPVDRGWDLERLYD...,GCCATCGTCAGCATGGCCTGCCGCTTCCCCGGCGGCATCGAGTCGC...,mal,A,mal
39,39,41,AB818354.1,BGC0000042.1,A1,GTVLITGGTGALGGHVARRLARNGAGHLVLTGRRGLGAPGAADLRD...,cmiP5,3,1537,"substrate mal, non-loading",CCCCCCC(N)CC(=O)C[CH][CH][C@H](O)CC(=O)[S],BGC0000042.1_cmiP5_mod3,IAIVGMACRFPGGVTSPEDLWQLVADGVDAVSSFPADRGWDVERLY...,CACCAGGACGTCGAGGCGTGCGCCGATCCTGGCCGTCCCGCCGGGT...,mal,A,mal
42,42,44,AB818354.1,BGC0000042.1,A1,GTTLVTGGTGGVGAETARLLARAGADHLLLTGRRGRAAPGMAELDD...,cmiP8,5,1503,"substrate mal, non-loading",CCCCCCC(N)CC(=O)C[CH][CH][C@H](O)CC(=O)CC(=O)C...,BGC0000042.1_cmiP8_mod5,IAIVAVGCRFPGGVRTPEDLWELVASGKDATTGFPENRGWDIGGIY...,CCGGGCGACCAGGGTCCGGAGGCGATCGACGATGCGGTGCTCCCGC...,mal,A,mal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1090,1091,1106,DQ983361.1,BGC0000158.1,A1,GTALITGGTGGLGAHVARWITERGAEHVVLVSRRGPDADGADLLRL...,ttnB,9,1771,"substrate mal, non-loading",CCC(=C[CH][CH][C@@H](C)[CH][C@@H](C)[CH]C[C@H]...,BGC0000158.1_ttnB_mod9,VVIVGMACRFPGGVMNPDDLWQLVADEADGVAPFPTDRGWDLGRLL...,CGAGCCGAGCAGGCCGTCCTCGATGATGCCCGCGGTGTGCACGACG...,mal,A,mal
1114,1115,1130,AY652953.1,BGC0000976.1,A1,GSYLITGGLGYLGLKVADWLVKQGVRHLILLSRSGLTKETQPAVEA...,CurG,4,1519,"substrate mal, non-loading",CC(=O)CC1=N[C@H]([C@H](O)CC(=O)[S])CS1,BGC0000976.1_CurG_mod4,IAIIGISGRYSGADDIEQFWQNVRDGVESICLFSEQELLNSGIEQE...,TTCAAGCTCTTGATTGGATGATTGCTTAATCTCGTCTAGAAGTTTA...,mal,A,mal
1140,1141,1156,AY310323.2,BGC0000034.1,A1,GTVLVTGGTGGLGAEAARWLARSGAAHLVLTGRRGPDAPGAAELRA...,fscD,12,1538,"substrate mal, non-loading",C[C@@H]([CH][C@H](C)[C@@H](O)[C@@H](C)C=[C]C=[...,BGC0000034.1_fscD_mod12,IVIVGMSCRYPGGVGSPDDLWRLVTEGTDAIGGLPVNRGWDLDSLY...,GTTGGCGGCGGCCTCGCCGATCGCGGGGGCGAGGGCGTCGAGCGCG...,mal,A,mal
1153,1154,1169,AY310323.2,BGC0000034.1,A2,GTVLVTGGTGTLGPHLARWLADQGARDLVLTSRRGADAPGAAELAA...,fscD,13,1507,"substrate mal, non-loading",C[C@@H]([CH][C@H](C)[C@@H](O)[C@@H](C)C=[C]C=[...,BGC0000034.1_fscD_mod13,IAIIGMSCRFPGGARSADAFWQLVADNTDAISEFPVNRGWDGDALY...,GGCGAGGGCGTCGAGGTAGGCGTTGGCGGCGGCGTAGTTGGCCTGG...,mal,A,mal


In [219]:
ccad_filtered

,Unnamed: 0,index,genbank_accession,mibig_accession,kr_type_annotation,kr_sequence,subunit_name,module_number,length_aa_residues,substrate_annotation,intermediate_smiles,name_in_fasta,protein_sequence,dna_sequence,substrate_type,new_kr,new_st
0,0,0,KX263301.1,BGC0001670.1,B1,GTVLIAGGTGMMGGLVAEHLVRAWSVRHLLLVSRQGPDAPDARDLA...,monAV,7,1801,"substrate mmal, non-loading",CCC([CH][CH]C=C(C)C(=O)[S])=C[C@@H](C)[CH][C]=...,BGC0001670.1_monAV_mod7,VAVVGMACRFPGGIETPEGLWELVAAGDDAIEPFPTDRGWDLEGIY...,GTGGCCGTCGTCGGCATGGCCTGCCGTTTCCCCGGCGGCATCGAGA...,mmal,B1,mmal
1,1,1,KX263301.1,BGC0001670.1,C1,GTVLITGPAGAPVADLAEHLVRTGQCRHLLLLPGDGELEEMAEELR...,monAI,1,1811,"substrate mmal, non-loading",CC(=O)[C@@H](C)C(=O)[S],BGC0001670.1_monAI_mod1,VAIVAMSCRYPGGAGTPEDLWRLVADGADAIGDFPTDRGWDLARLF...,GTCGCCATCGTGGCCATGAGCTGCCGCTATCCCGGCGGCGCAGGCA...,mmal,C1,mmal
2,2,2,KX458106.1,BGC0001690.1,B1,GTVLVTGGTGGLGRVMARHLVVEHGVRNLLLVSRRGPAAEGAEELV...,SgnS1,3,1720,"substrate mal, non-loading",CC(=O)C[C@@H](O)CC=[C]C(=O)[S],BGC0001690.1_SgnS1_mod3,VAIIGMACRFPGGVHSPEALWRLLAEGGDAITPMPADRGWDLDRLY...,GTGGCGATCATCGGCATGGCCTGCCGCTTCCCCGGCGGCGTGCACT...,mal,B,mal
3,3,3,KX458106.1,BGC0001690.1,A1,GTVLITGGTGALGAEAARWLARSGAEHLLLTSRRGPEAPGAAELAA...,SgnS2,7,1522,"substrate mal, non-loading",CC(=O)C[C@@H](O)CC=[C]C=[C]C=[C]C=[C][C@H](O)C...,BGC0001690.1_SgnS2_mod7,IVIVGMSCRYPGGVASPEDLWRLVSEGTDAISGLPTDRGWDLDALY...,ATCGTCATCGTCGGCATGAGCTGCCGCTACCCGGGCGGCGTCGCCT...,mal,A,mal
4,4,4,KX263301.1,BGC0001670.1,B1,GTVLITGGTGVLAAAVAEHLVREWGVRHLLLAGRRGSEAPGSSELA...,monAIII,4,2068,"substrate mmal, non-loading",CC(=O)[C@@H](C)[CH][C@H](C)C=[C][CH][C@H](C)C(...,BGC0001670.1_monAIII_mod4,IAIVGMACRYPGGVTSPEQLWQLVATGTDAIGPFPEDRGWDTAGLF...,ATCGCCATCGTCGGCATGGCGTGCCGCTACCCGGGCGGCGTGACCT...,mmal,B1,mmal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1162,1163,1179,CP004025.1,BGC0001394.1,A1,ATYLVTGGLGGLGLKVARWLVEQGARHLVLVGRRALTAGGEESKRR...,polyketidesynthase_3,3,1763,"substrate mmal, non-loading",CC(=C[C@@H](C)[CH]Cc1ccccc1)C=C(C)C(=O)[S],BGC0001394.1_polyketidesynthase_3_mod3,VALIGMACRFPGGASDPESFWHVLRDGVDAVTEVPSSRWTQEEAAR...,TGCCTGCGCGCGCGGCGGCGGTGCTGCGTCGCCCGGCAGCGGTATC...,mmal,A1,mmal
1163,1164,1180,CP000667.1,BGC0000142.1,B1,GTVLVTGGTGVLGALVARHLVVGHGVRELVLASRRGLAAPGAVELR...,beta-ketoacylsynthase,2,1711,"substrate mal, non-loading",CC(N)CC=[C]C=[C]C(=O)[S],BGC0000142.1_beta-ketoacylsynthase_mod2,IAIVAMGCRFPGGVRSPEDLWELLEGGVDAVSGFPTDRGWDLVGMY...,CGCGACAGCAGCCCGGACCAGATCAAGCAGCATCCGATGCTGCTCC...,mal,B,mal
1164,1165,1181,CP000667.1,BGC0000142.1,B1,GTVLVTGGTGVLGALVARHLVVGHGVRELVLASRRGLAAPGAVELR...,beta-ketoacylsynthase,2,1711,"substrate mal, non-loading",CC(N)CC=[C]C=[C]C(=O)[S],BGC0000142.1_beta-ketoacylsynthase_mod2,IAIVAMGCRFPGGVRSPEDLWELLEGGVDAVSGFPTDRGWDLVGMY...,CGCGACAGCAGCCCGGACCAGATCAAGCAGCATCCGATGCTGCTCC...,mal,B,mal
1165,1166,1182,CP000667.1,BGC0000142.1,B1,GTVLVTGGTGGLGRLVAEHLITRHGVRRLLLASRRGPAAPGVDELV...,beta-ketoacylsynthase,4,1746,"substrate mal, non-loading",CC(C=[C]C(=O)[S])=C[C]=C[C]=CCC(C)N,BGC0000142.1_beta-ketoacylsynthase_mod4,VAIVAMGCRFPGGVRSPEDLWELLEGGVDAVSGFPTDRGWDLVGMY...,GGCAGCGACGTGGACGCGGATCAGGTCGACGGTGACCCGGTCCCGC...,mal,B,mal


In [220]:
ccad_filtered

,Unnamed: 0,index,genbank_accession,mibig_accession,kr_type_annotation,kr_sequence,subunit_name,module_number,length_aa_residues,substrate_annotation,intermediate_smiles,name_in_fasta,protein_sequence,dna_sequence,substrate_type,new_kr,new_st
0,0,0,KX263301.1,BGC0001670.1,B1,GTVLIAGGTGMMGGLVAEHLVRAWSVRHLLLVSRQGPDAPDARDLA...,monAV,7,1801,"substrate mmal, non-loading",CCC([CH][CH]C=C(C)C(=O)[S])=C[C@@H](C)[CH][C]=...,BGC0001670.1_monAV_mod7,VAVVGMACRFPGGIETPEGLWELVAAGDDAIEPFPTDRGWDLEGIY...,GTGGCCGTCGTCGGCATGGCCTGCCGTTTCCCCGGCGGCATCGAGA...,mmal,B1,mmal
1,1,1,KX263301.1,BGC0001670.1,C1,GTVLITGPAGAPVADLAEHLVRTGQCRHLLLLPGDGELEEMAEELR...,monAI,1,1811,"substrate mmal, non-loading",CC(=O)[C@@H](C)C(=O)[S],BGC0001670.1_monAI_mod1,VAIVAMSCRYPGGAGTPEDLWRLVADGADAIGDFPTDRGWDLARLF...,GTCGCCATCGTGGCCATGAGCTGCCGCTATCCCGGCGGCGCAGGCA...,mmal,C1,mmal
2,2,2,KX458106.1,BGC0001690.1,B1,GTVLVTGGTGGLGRVMARHLVVEHGVRNLLLVSRRGPAAEGAEELV...,SgnS1,3,1720,"substrate mal, non-loading",CC(=O)C[C@@H](O)CC=[C]C(=O)[S],BGC0001690.1_SgnS1_mod3,VAIIGMACRFPGGVHSPEALWRLLAEGGDAITPMPADRGWDLDRLY...,GTGGCGATCATCGGCATGGCCTGCCGCTTCCCCGGCGGCGTGCACT...,mal,B,mal
3,3,3,KX458106.1,BGC0001690.1,A1,GTVLITGGTGALGAEAARWLARSGAEHLLLTSRRGPEAPGAAELAA...,SgnS2,7,1522,"substrate mal, non-loading",CC(=O)C[C@@H](O)CC=[C]C=[C]C=[C]C=[C][C@H](O)C...,BGC0001690.1_SgnS2_mod7,IVIVGMSCRYPGGVASPEDLWRLVSEGTDAISGLPTDRGWDLDALY...,ATCGTCATCGTCGGCATGAGCTGCCGCTACCCGGGCGGCGTCGCCT...,mal,A,mal
4,4,4,KX263301.1,BGC0001670.1,B1,GTVLITGGTGVLAAAVAEHLVREWGVRHLLLAGRRGSEAPGSSELA...,monAIII,4,2068,"substrate mmal, non-loading",CC(=O)[C@@H](C)[CH][C@H](C)C=[C][CH][C@H](C)C(...,BGC0001670.1_monAIII_mod4,IAIVGMACRYPGGVTSPEQLWQLVATGTDAIGPFPEDRGWDTAGLF...,ATCGCCATCGTCGGCATGGCGTGCCGCTACCCGGGCGGCGTGACCT...,mmal,B1,mmal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1162,1163,1179,CP004025.1,BGC0001394.1,A1,ATYLVTGGLGGLGLKVARWLVEQGARHLVLVGRRALTAGGEESKRR...,polyketidesynthase_3,3,1763,"substrate mmal, non-loading",CC(=C[C@@H](C)[CH]Cc1ccccc1)C=C(C)C(=O)[S],BGC0001394.1_polyketidesynthase_3_mod3,VALIGMACRFPGGASDPESFWHVLRDGVDAVTEVPSSRWTQEEAAR...,TGCCTGCGCGCGCGGCGGCGGTGCTGCGTCGCCCGGCAGCGGTATC...,mmal,A1,mmal
1163,1164,1180,CP000667.1,BGC0000142.1,B1,GTVLVTGGTGVLGALVARHLVVGHGVRELVLASRRGLAAPGAVELR...,beta-ketoacylsynthase,2,1711,"substrate mal, non-loading",CC(N)CC=[C]C=[C]C(=O)[S],BGC0000142.1_beta-ketoacylsynthase_mod2,IAIVAMGCRFPGGVRSPEDLWELLEGGVDAVSGFPTDRGWDLVGMY...,CGCGACAGCAGCCCGGACCAGATCAAGCAGCATCCGATGCTGCTCC...,mal,B,mal
1164,1165,1181,CP000667.1,BGC0000142.1,B1,GTVLVTGGTGVLGALVARHLVVGHGVRELVLASRRGLAAPGAVELR...,beta-ketoacylsynthase,2,1711,"substrate mal, non-loading",CC(N)CC=[C]C=[C]C(=O)[S],BGC0000142.1_beta-ketoacylsynthase_mod2,IAIVAMGCRFPGGVRSPEDLWELLEGGVDAVSGFPTDRGWDLVGMY...,CGCGACAGCAGCCCGGACCAGATCAAGCAGCATCCGATGCTGCTCC...,mal,B,mal
1165,1166,1182,CP000667.1,BGC0000142.1,B1,GTVLVTGGTGGLGRLVAEHLITRHGVRRLLLASRRGPAAPGVDELV...,beta-ketoacylsynthase,4,1746,"substrate mal, non-loading",CC(C=[C]C(=O)[S])=C[C]=C[C]=CCC(C)N,BGC0000142.1_beta-ketoacylsynthase_mod4,VAIVAMGCRFPGGVRSPEDLWELLEGGVDAVSGFPTDRGWDLVGMY...,GGCAGCGACGTGGACGCGGATCAGGTCGACGGTGACCCGGTCCCGC...,mal,B,mal


In [221]:
'''
#cut 600 b1

# Identify all B1 rows
a_mask = xiang_filtered["Annotation"] == "A"
a_indices = xiang_filtered[a_mask].index

# Randomly sample 600 B1 indices to drop
#b1_to_remove = np.random.choice(b1_indices, size=100, replace=False)

# Drop those rows from the DataFrame
xiang_filtered = xiang_filtered.drop(index=a_indices)
'''

'\n#cut 600 b1\n\n# Identify all B1 rows\na_mask = xiang_filtered["Annotation"] == "A"\na_indices = xiang_filtered[a_mask].index\n\n# Randomly sample 600 B1 indices to drop\n#b1_to_remove = np.random.choice(b1_indices, size=100, replace=False)\n\n# Drop those rows from the DataFrame\nxiang_filtered = xiang_filtered.drop(index=a_indices)\n'

In [222]:
ccad_filtered

,Unnamed: 0,index,genbank_accession,mibig_accession,kr_type_annotation,kr_sequence,subunit_name,module_number,length_aa_residues,substrate_annotation,intermediate_smiles,name_in_fasta,protein_sequence,dna_sequence,substrate_type,new_kr,new_st
0,0,0,KX263301.1,BGC0001670.1,B1,GTVLIAGGTGMMGGLVAEHLVRAWSVRHLLLVSRQGPDAPDARDLA...,monAV,7,1801,"substrate mmal, non-loading",CCC([CH][CH]C=C(C)C(=O)[S])=C[C@@H](C)[CH][C]=...,BGC0001670.1_monAV_mod7,VAVVGMACRFPGGIETPEGLWELVAAGDDAIEPFPTDRGWDLEGIY...,GTGGCCGTCGTCGGCATGGCCTGCCGTTTCCCCGGCGGCATCGAGA...,mmal,B1,mmal
1,1,1,KX263301.1,BGC0001670.1,C1,GTVLITGPAGAPVADLAEHLVRTGQCRHLLLLPGDGELEEMAEELR...,monAI,1,1811,"substrate mmal, non-loading",CC(=O)[C@@H](C)C(=O)[S],BGC0001670.1_monAI_mod1,VAIVAMSCRYPGGAGTPEDLWRLVADGADAIGDFPTDRGWDLARLF...,GTCGCCATCGTGGCCATGAGCTGCCGCTATCCCGGCGGCGCAGGCA...,mmal,C1,mmal
2,2,2,KX458106.1,BGC0001690.1,B1,GTVLVTGGTGGLGRVMARHLVVEHGVRNLLLVSRRGPAAEGAEELV...,SgnS1,3,1720,"substrate mal, non-loading",CC(=O)C[C@@H](O)CC=[C]C(=O)[S],BGC0001690.1_SgnS1_mod3,VAIIGMACRFPGGVHSPEALWRLLAEGGDAITPMPADRGWDLDRLY...,GTGGCGATCATCGGCATGGCCTGCCGCTTCCCCGGCGGCGTGCACT...,mal,B,mal
3,3,3,KX458106.1,BGC0001690.1,A1,GTVLITGGTGALGAEAARWLARSGAEHLLLTSRRGPEAPGAAELAA...,SgnS2,7,1522,"substrate mal, non-loading",CC(=O)C[C@@H](O)CC=[C]C=[C]C=[C]C=[C][C@H](O)C...,BGC0001690.1_SgnS2_mod7,IVIVGMSCRYPGGVASPEDLWRLVSEGTDAISGLPTDRGWDLDALY...,ATCGTCATCGTCGGCATGAGCTGCCGCTACCCGGGCGGCGTCGCCT...,mal,A,mal
4,4,4,KX263301.1,BGC0001670.1,B1,GTVLITGGTGVLAAAVAEHLVREWGVRHLLLAGRRGSEAPGSSELA...,monAIII,4,2068,"substrate mmal, non-loading",CC(=O)[C@@H](C)[CH][C@H](C)C=[C][CH][C@H](C)C(...,BGC0001670.1_monAIII_mod4,IAIVGMACRYPGGVTSPEQLWQLVATGTDAIGPFPEDRGWDTAGLF...,ATCGCCATCGTCGGCATGGCGTGCCGCTACCCGGGCGGCGTGACCT...,mmal,B1,mmal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1162,1163,1179,CP004025.1,BGC0001394.1,A1,ATYLVTGGLGGLGLKVARWLVEQGARHLVLVGRRALTAGGEESKRR...,polyketidesynthase_3,3,1763,"substrate mmal, non-loading",CC(=C[C@@H](C)[CH]Cc1ccccc1)C=C(C)C(=O)[S],BGC0001394.1_polyketidesynthase_3_mod3,VALIGMACRFPGGASDPESFWHVLRDGVDAVTEVPSSRWTQEEAAR...,TGCCTGCGCGCGCGGCGGCGGTGCTGCGTCGCCCGGCAGCGGTATC...,mmal,A1,mmal
1163,1164,1180,CP000667.1,BGC0000142.1,B1,GTVLVTGGTGVLGALVARHLVVGHGVRELVLASRRGLAAPGAVELR...,beta-ketoacylsynthase,2,1711,"substrate mal, non-loading",CC(N)CC=[C]C=[C]C(=O)[S],BGC0000142.1_beta-ketoacylsynthase_mod2,IAIVAMGCRFPGGVRSPEDLWELLEGGVDAVSGFPTDRGWDLVGMY...,CGCGACAGCAGCCCGGACCAGATCAAGCAGCATCCGATGCTGCTCC...,mal,B,mal
1164,1165,1181,CP000667.1,BGC0000142.1,B1,GTVLVTGGTGVLGALVARHLVVGHGVRELVLASRRGLAAPGAVELR...,beta-ketoacylsynthase,2,1711,"substrate mal, non-loading",CC(N)CC=[C]C=[C]C(=O)[S],BGC0000142.1_beta-ketoacylsynthase_mod2,IAIVAMGCRFPGGVRSPEDLWELLEGGVDAVSGFPTDRGWDLVGMY...,CGCGACAGCAGCCCGGACCAGATCAAGCAGCATCCGATGCTGCTCC...,mal,B,mal
1165,1166,1182,CP000667.1,BGC0000142.1,B1,GTVLVTGGTGGLGRLVAEHLITRHGVRRLLLASRRGPAAPGVDELV...,beta-ketoacylsynthase,4,1746,"substrate mal, non-loading",CC(C=[C]C(=O)[S])=C[C]=C[C]=CCC(C)N,BGC0000142.1_beta-ketoacylsynthase_mod4,VAIVAMGCRFPGGVRSPEDLWELLEGGVDAVSGFPTDRGWDLVGMY...,GGCAGCGACGTGGACGCGGATCAGGTCGACGGTGACCCGGTCCCGC...,mal,B,mal


In [223]:
ccad_filtered

,Unnamed: 0,index,genbank_accession,mibig_accession,kr_type_annotation,kr_sequence,subunit_name,module_number,length_aa_residues,substrate_annotation,intermediate_smiles,name_in_fasta,protein_sequence,dna_sequence,substrate_type,new_kr,new_st
0,0,0,KX263301.1,BGC0001670.1,B1,GTVLIAGGTGMMGGLVAEHLVRAWSVRHLLLVSRQGPDAPDARDLA...,monAV,7,1801,"substrate mmal, non-loading",CCC([CH][CH]C=C(C)C(=O)[S])=C[C@@H](C)[CH][C]=...,BGC0001670.1_monAV_mod7,VAVVGMACRFPGGIETPEGLWELVAAGDDAIEPFPTDRGWDLEGIY...,GTGGCCGTCGTCGGCATGGCCTGCCGTTTCCCCGGCGGCATCGAGA...,mmal,B1,mmal
1,1,1,KX263301.1,BGC0001670.1,C1,GTVLITGPAGAPVADLAEHLVRTGQCRHLLLLPGDGELEEMAEELR...,monAI,1,1811,"substrate mmal, non-loading",CC(=O)[C@@H](C)C(=O)[S],BGC0001670.1_monAI_mod1,VAIVAMSCRYPGGAGTPEDLWRLVADGADAIGDFPTDRGWDLARLF...,GTCGCCATCGTGGCCATGAGCTGCCGCTATCCCGGCGGCGCAGGCA...,mmal,C1,mmal
2,2,2,KX458106.1,BGC0001690.1,B1,GTVLVTGGTGGLGRVMARHLVVEHGVRNLLLVSRRGPAAEGAEELV...,SgnS1,3,1720,"substrate mal, non-loading",CC(=O)C[C@@H](O)CC=[C]C(=O)[S],BGC0001690.1_SgnS1_mod3,VAIIGMACRFPGGVHSPEALWRLLAEGGDAITPMPADRGWDLDRLY...,GTGGCGATCATCGGCATGGCCTGCCGCTTCCCCGGCGGCGTGCACT...,mal,B,mal
3,3,3,KX458106.1,BGC0001690.1,A1,GTVLITGGTGALGAEAARWLARSGAEHLLLTSRRGPEAPGAAELAA...,SgnS2,7,1522,"substrate mal, non-loading",CC(=O)C[C@@H](O)CC=[C]C=[C]C=[C]C=[C][C@H](O)C...,BGC0001690.1_SgnS2_mod7,IVIVGMSCRYPGGVASPEDLWRLVSEGTDAISGLPTDRGWDLDALY...,ATCGTCATCGTCGGCATGAGCTGCCGCTACCCGGGCGGCGTCGCCT...,mal,A,mal
4,4,4,KX263301.1,BGC0001670.1,B1,GTVLITGGTGVLAAAVAEHLVREWGVRHLLLAGRRGSEAPGSSELA...,monAIII,4,2068,"substrate mmal, non-loading",CC(=O)[C@@H](C)[CH][C@H](C)C=[C][CH][C@H](C)C(...,BGC0001670.1_monAIII_mod4,IAIVGMACRYPGGVTSPEQLWQLVATGTDAIGPFPEDRGWDTAGLF...,ATCGCCATCGTCGGCATGGCGTGCCGCTACCCGGGCGGCGTGACCT...,mmal,B1,mmal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1162,1163,1179,CP004025.1,BGC0001394.1,A1,ATYLVTGGLGGLGLKVARWLVEQGARHLVLVGRRALTAGGEESKRR...,polyketidesynthase_3,3,1763,"substrate mmal, non-loading",CC(=C[C@@H](C)[CH]Cc1ccccc1)C=C(C)C(=O)[S],BGC0001394.1_polyketidesynthase_3_mod3,VALIGMACRFPGGASDPESFWHVLRDGVDAVTEVPSSRWTQEEAAR...,TGCCTGCGCGCGCGGCGGCGGTGCTGCGTCGCCCGGCAGCGGTATC...,mmal,A1,mmal
1163,1164,1180,CP000667.1,BGC0000142.1,B1,GTVLVTGGTGVLGALVARHLVVGHGVRELVLASRRGLAAPGAVELR...,beta-ketoacylsynthase,2,1711,"substrate mal, non-loading",CC(N)CC=[C]C=[C]C(=O)[S],BGC0000142.1_beta-ketoacylsynthase_mod2,IAIVAMGCRFPGGVRSPEDLWELLEGGVDAVSGFPTDRGWDLVGMY...,CGCGACAGCAGCCCGGACCAGATCAAGCAGCATCCGATGCTGCTCC...,mal,B,mal
1164,1165,1181,CP000667.1,BGC0000142.1,B1,GTVLVTGGTGVLGALVARHLVVGHGVRELVLASRRGLAAPGAVELR...,beta-ketoacylsynthase,2,1711,"substrate mal, non-loading",CC(N)CC=[C]C=[C]C(=O)[S],BGC0000142.1_beta-ketoacylsynthase_mod2,IAIVAMGCRFPGGVRSPEDLWELLEGGVDAVSGFPTDRGWDLVGMY...,CGCGACAGCAGCCCGGACCAGATCAAGCAGCATCCGATGCTGCTCC...,mal,B,mal
1165,1166,1182,CP000667.1,BGC0000142.1,B1,GTVLVTGGTGGLGRLVAEHLITRHGVRRLLLASRRGPAAPGVDELV...,beta-ketoacylsynthase,4,1746,"substrate mal, non-loading",CC(C=[C]C(=O)[S])=C[C]=C[C]=CCC(C)N,BGC0000142.1_beta-ketoacylsynthase_mod4,VAIVAMGCRFPGGVRSPEDLWELLEGGVDAVSGFPTDRGWDLVGMY...,GGCAGCGACGTGGACGCGGATCAGGTCGACGGTGACCCGGTCCCGC...,mal,B,mal


In [224]:
clustercad_idx = ccad_filtered.index.to_numpy().tolist()
ccad_filtered_embeddings = [ccad_filtered_embeddings[i] for i in clustercad_idx]
print(len(ccad_filtered_embeddings))

1157


In [225]:
ccad_filtered

,Unnamed: 0,index,genbank_accession,mibig_accession,kr_type_annotation,kr_sequence,subunit_name,module_number,length_aa_residues,substrate_annotation,intermediate_smiles,name_in_fasta,protein_sequence,dna_sequence,substrate_type,new_kr,new_st
0,0,0,KX263301.1,BGC0001670.1,B1,GTVLIAGGTGMMGGLVAEHLVRAWSVRHLLLVSRQGPDAPDARDLA...,monAV,7,1801,"substrate mmal, non-loading",CCC([CH][CH]C=C(C)C(=O)[S])=C[C@@H](C)[CH][C]=...,BGC0001670.1_monAV_mod7,VAVVGMACRFPGGIETPEGLWELVAAGDDAIEPFPTDRGWDLEGIY...,GTGGCCGTCGTCGGCATGGCCTGCCGTTTCCCCGGCGGCATCGAGA...,mmal,B1,mmal
1,1,1,KX263301.1,BGC0001670.1,C1,GTVLITGPAGAPVADLAEHLVRTGQCRHLLLLPGDGELEEMAEELR...,monAI,1,1811,"substrate mmal, non-loading",CC(=O)[C@@H](C)C(=O)[S],BGC0001670.1_monAI_mod1,VAIVAMSCRYPGGAGTPEDLWRLVADGADAIGDFPTDRGWDLARLF...,GTCGCCATCGTGGCCATGAGCTGCCGCTATCCCGGCGGCGCAGGCA...,mmal,C1,mmal
2,2,2,KX458106.1,BGC0001690.1,B1,GTVLVTGGTGGLGRVMARHLVVEHGVRNLLLVSRRGPAAEGAEELV...,SgnS1,3,1720,"substrate mal, non-loading",CC(=O)C[C@@H](O)CC=[C]C(=O)[S],BGC0001690.1_SgnS1_mod3,VAIIGMACRFPGGVHSPEALWRLLAEGGDAITPMPADRGWDLDRLY...,GTGGCGATCATCGGCATGGCCTGCCGCTTCCCCGGCGGCGTGCACT...,mal,B,mal
3,3,3,KX458106.1,BGC0001690.1,A1,GTVLITGGTGALGAEAARWLARSGAEHLLLTSRRGPEAPGAAELAA...,SgnS2,7,1522,"substrate mal, non-loading",CC(=O)C[C@@H](O)CC=[C]C=[C]C=[C]C=[C][C@H](O)C...,BGC0001690.1_SgnS2_mod7,IVIVGMSCRYPGGVASPEDLWRLVSEGTDAISGLPTDRGWDLDALY...,ATCGTCATCGTCGGCATGAGCTGCCGCTACCCGGGCGGCGTCGCCT...,mal,A,mal
4,4,4,KX263301.1,BGC0001670.1,B1,GTVLITGGTGVLAAAVAEHLVREWGVRHLLLAGRRGSEAPGSSELA...,monAIII,4,2068,"substrate mmal, non-loading",CC(=O)[C@@H](C)[CH][C@H](C)C=[C][CH][C@H](C)C(...,BGC0001670.1_monAIII_mod4,IAIVGMACRYPGGVTSPEQLWQLVATGTDAIGPFPEDRGWDTAGLF...,ATCGCCATCGTCGGCATGGCGTGCCGCTACCCGGGCGGCGTGACCT...,mmal,B1,mmal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1162,1163,1179,CP004025.1,BGC0001394.1,A1,ATYLVTGGLGGLGLKVARWLVEQGARHLVLVGRRALTAGGEESKRR...,polyketidesynthase_3,3,1763,"substrate mmal, non-loading",CC(=C[C@@H](C)[CH]Cc1ccccc1)C=C(C)C(=O)[S],BGC0001394.1_polyketidesynthase_3_mod3,VALIGMACRFPGGASDPESFWHVLRDGVDAVTEVPSSRWTQEEAAR...,TGCCTGCGCGCGCGGCGGCGGTGCTGCGTCGCCCGGCAGCGGTATC...,mmal,A1,mmal
1163,1164,1180,CP000667.1,BGC0000142.1,B1,GTVLVTGGTGVLGALVARHLVVGHGVRELVLASRRGLAAPGAVELR...,beta-ketoacylsynthase,2,1711,"substrate mal, non-loading",CC(N)CC=[C]C=[C]C(=O)[S],BGC0000142.1_beta-ketoacylsynthase_mod2,IAIVAMGCRFPGGVRSPEDLWELLEGGVDAVSGFPTDRGWDLVGMY...,CGCGACAGCAGCCCGGACCAGATCAAGCAGCATCCGATGCTGCTCC...,mal,B,mal
1164,1165,1181,CP000667.1,BGC0000142.1,B1,GTVLVTGGTGVLGALVARHLVVGHGVRELVLASRRGLAAPGAVELR...,beta-ketoacylsynthase,2,1711,"substrate mal, non-loading",CC(N)CC=[C]C=[C]C(=O)[S],BGC0000142.1_beta-ketoacylsynthase_mod2,IAIVAMGCRFPGGVRSPEDLWELLEGGVDAVSGFPTDRGWDLVGMY...,CGCGACAGCAGCCCGGACCAGATCAAGCAGCATCCGATGCTGCTCC...,mal,B,mal
1165,1166,1182,CP000667.1,BGC0000142.1,B1,GTVLVTGGTGGLGRLVAEHLITRHGVRRLLLASRRGPAAPGVDELV...,beta-ketoacylsynthase,4,1746,"substrate mal, non-loading",CC(C=[C]C(=O)[S])=C[C]=C[C]=CCC(C)N,BGC0000142.1_beta-ketoacylsynthase_mod4,VAIVAMGCRFPGGVRSPEDLWELLEGGVDAVSGFPTDRGWDLVGMY...,GGCAGCGACGTGGACGCGGATCAGGTCGACGGTGACCCGGTCCCGC...,mal,B,mal


In [226]:
ccad_filtered = ccad_filtered.reset_index()


In [227]:
print(len(ccad_filtered))
print(len(ccad_filtered_embeddings))

1157
1157


In [228]:
sum(ccad_filtered["new_kr"] == "B")

492

In [229]:
#xiang_idx

In [230]:
len(xiang_filtered_embeddings)

2

In [231]:

print(len(ccad_filtered[ccad_filtered["new_kr"] == "B1"]))


280


In [232]:
#average pooling. 
ccad_filtered_embeddings = [x.mean(dim=1).squeeze(0) for x in ccad_filtered_embeddings]

In [233]:

print(ccad_filtered_embeddings[0].shape)


torch.Size([1536])


In [234]:
len(xiang_filtered_embeddings)

2

In [235]:
#stacking

ccad_embeddings = torch.stack(ccad_filtered_embeddings)

In [236]:

#this is standarization but we should do it AFTER -- on training only. 
'''
# After stacking embeddings
print(f"Before normalization - mean: {xiang_embeddings.mean():.3f}, std: {xiang_embeddings.std():.3f}")

# Standardize features
mu = xiang_embeddings.mean(0)
sigma = xiang_embeddings.std(0) + 1e-9
xiang_embeddings = (xiang_embeddings - mu) / sigma

print(f"After normalization - mean: {xiang_embeddings.mean():.3f}, std: {xiang_embeddings.std():.3f}")
'''

'\n# After stacking embeddings\nprint(f"Before normalization - mean: {xiang_embeddings.mean():.3f}, std: {xiang_embeddings.std():.3f}")\n\n# Standardize features\nmu = xiang_embeddings.mean(0)\nsigma = xiang_embeddings.std(0) + 1e-9\nxiang_embeddings = (xiang_embeddings - mu) / sigma\n\nprint(f"After normalization - mean: {xiang_embeddings.mean():.3f}, std: {xiang_embeddings.std():.3f}")\n'

In [237]:
annotations_unique = ccad_filtered["new_kr"].unique()

In [238]:
annotations_unique

array(['B1', 'C1', 'B', 'A', 'A2', 'A1', 'C', 'B2', 'C2'], dtype=object)

In [239]:
#outputs need to be converted to numerical values.

annotations_unique = ccad_filtered["new_kr"].unique()
annotations_unique.sort() #sort alphabetically 

annotation_enumerated = {x: i for i, x in enumerate(annotations_unique)}
print(annotation_enumerated)

'''
annotations_unique_cc = ccad_filtered["kr_type_annotation"].unique()
annotations_unique_cc.sort() #sort alphabetically 

annotation_enumerated_cc = {x: i for i, x in enumerate(annotations_unique_cc)}
print(annotation_enumerated_cc)
'''

{'A': 0, 'A1': 1, 'A2': 2, 'B': 3, 'B1': 4, 'B2': 5, 'C': 6, 'C1': 7, 'C2': 8}


'\nannotations_unique_cc = ccad_filtered["kr_type_annotation"].unique()\nannotations_unique_cc.sort() #sort alphabetically \n\nannotation_enumerated_cc = {x: i for i, x in enumerate(annotations_unique_cc)}\nprint(annotation_enumerated_cc)\n'

In [240]:
ccad_filtered

,level_0,Unnamed: 0,index,genbank_accession,mibig_accession,kr_type_annotation,kr_sequence,subunit_name,module_number,length_aa_residues,substrate_annotation,intermediate_smiles,name_in_fasta,protein_sequence,dna_sequence,substrate_type,new_kr,new_st
0,0,0,0,KX263301.1,BGC0001670.1,B1,GTVLIAGGTGMMGGLVAEHLVRAWSVRHLLLVSRQGPDAPDARDLA...,monAV,7,1801,"substrate mmal, non-loading",CCC([CH][CH]C=C(C)C(=O)[S])=C[C@@H](C)[CH][C]=...,BGC0001670.1_monAV_mod7,VAVVGMACRFPGGIETPEGLWELVAAGDDAIEPFPTDRGWDLEGIY...,GTGGCCGTCGTCGGCATGGCCTGCCGTTTCCCCGGCGGCATCGAGA...,mmal,B1,mmal
1,1,1,1,KX263301.1,BGC0001670.1,C1,GTVLITGPAGAPVADLAEHLVRTGQCRHLLLLPGDGELEEMAEELR...,monAI,1,1811,"substrate mmal, non-loading",CC(=O)[C@@H](C)C(=O)[S],BGC0001670.1_monAI_mod1,VAIVAMSCRYPGGAGTPEDLWRLVADGADAIGDFPTDRGWDLARLF...,GTCGCCATCGTGGCCATGAGCTGCCGCTATCCCGGCGGCGCAGGCA...,mmal,C1,mmal
2,2,2,2,KX458106.1,BGC0001690.1,B1,GTVLVTGGTGGLGRVMARHLVVEHGVRNLLLVSRRGPAAEGAEELV...,SgnS1,3,1720,"substrate mal, non-loading",CC(=O)C[C@@H](O)CC=[C]C(=O)[S],BGC0001690.1_SgnS1_mod3,VAIIGMACRFPGGVHSPEALWRLLAEGGDAITPMPADRGWDLDRLY...,GTGGCGATCATCGGCATGGCCTGCCGCTTCCCCGGCGGCGTGCACT...,mal,B,mal
3,3,3,3,KX458106.1,BGC0001690.1,A1,GTVLITGGTGALGAEAARWLARSGAEHLLLTSRRGPEAPGAAELAA...,SgnS2,7,1522,"substrate mal, non-loading",CC(=O)C[C@@H](O)CC=[C]C=[C]C=[C]C=[C][C@H](O)C...,BGC0001690.1_SgnS2_mod7,IVIVGMSCRYPGGVASPEDLWRLVSEGTDAISGLPTDRGWDLDALY...,ATCGTCATCGTCGGCATGAGCTGCCGCTACCCGGGCGGCGTCGCCT...,mal,A,mal
4,4,4,4,KX263301.1,BGC0001670.1,B1,GTVLITGGTGVLAAAVAEHLVREWGVRHLLLAGRRGSEAPGSSELA...,monAIII,4,2068,"substrate mmal, non-loading",CC(=O)[C@@H](C)[CH][C@H](C)C=[C][CH][C@H](C)C(...,BGC0001670.1_monAIII_mod4,IAIVGMACRYPGGVTSPEQLWQLVATGTDAIGPFPEDRGWDTAGLF...,ATCGCCATCGTCGGCATGGCGTGCCGCTACCCGGGCGGCGTGACCT...,mmal,B1,mmal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1152,1162,1163,1179,CP004025.1,BGC0001394.1,A1,ATYLVTGGLGGLGLKVARWLVEQGARHLVLVGRRALTAGGEESKRR...,polyketidesynthase_3,3,1763,"substrate mmal, non-loading",CC(=C[C@@H](C)[CH]Cc1ccccc1)C=C(C)C(=O)[S],BGC0001394.1_polyketidesynthase_3_mod3,VALIGMACRFPGGASDPESFWHVLRDGVDAVTEVPSSRWTQEEAAR...,TGCCTGCGCGCGCGGCGGCGGTGCTGCGTCGCCCGGCAGCGGTATC...,mmal,A1,mmal
1153,1163,1164,1180,CP000667.1,BGC0000142.1,B1,GTVLVTGGTGVLGALVARHLVVGHGVRELVLASRRGLAAPGAVELR...,beta-ketoacylsynthase,2,1711,"substrate mal, non-loading",CC(N)CC=[C]C=[C]C(=O)[S],BGC0000142.1_beta-ketoacylsynthase_mod2,IAIVAMGCRFPGGVRSPEDLWELLEGGVDAVSGFPTDRGWDLVGMY...,CGCGACAGCAGCCCGGACCAGATCAAGCAGCATCCGATGCTGCTCC...,mal,B,mal
1154,1164,1165,1181,CP000667.1,BGC0000142.1,B1,GTVLVTGGTGVLGALVARHLVVGHGVRELVLASRRGLAAPGAVELR...,beta-ketoacylsynthase,2,1711,"substrate mal, non-loading",CC(N)CC=[C]C=[C]C(=O)[S],BGC0000142.1_beta-ketoacylsynthase_mod2,IAIVAMGCRFPGGVRSPEDLWELLEGGVDAVSGFPTDRGWDLVGMY...,CGCGACAGCAGCCCGGACCAGATCAAGCAGCATCCGATGCTGCTCC...,mal,B,mal
1155,1165,1166,1182,CP000667.1,BGC0000142.1,B1,GTVLVTGGTGGLGRLVAEHLITRHGVRRLLLASRRGPAAPGVDELV...,beta-ketoacylsynthase,4,1746,"substrate mal, non-loading",CC(C=[C]C(=O)[S])=C[C]=C[C]=CCC(C)N,BGC0000142.1_beta-ketoacylsynthase_mod4,VAIVAMGCRFPGGVRSPEDLWELLEGGVDAVSGFPTDRGWDLVGMY...,GGCAGCGACGTGGACGCGGATCAGGTCGACGGTGACCCGGTCCCGC...,mal,B,mal


In [241]:
#xiang_filtered["AnnotationEnumerated"] = xiang_filtered["Annotation"].map(annotation_enumerated)
ccad_filtered["AnnotationEnumerated"] = ccad_filtered["new_kr"].map(annotation_enumerated)
#ccad_filtered['AnnotationEnumerated'] = ccad_filtered['AnnotationEnumerated'].astype(np.int64)

In [242]:
annotation_enumerated

{'A': 0, 'A1': 1, 'A2': 2, 'B': 3, 'B1': 4, 'B2': 5, 'C': 6, 'C1': 7, 'C2': 8}

In [243]:
#print(xiang_filtered["AnnotationEnumerated"])
print(ccad_filtered["AnnotationEnumerated"])

0       4
1       7
2       3
3       0
4       4
       ..
1152    1
1153    3
1154    3
1155    3
1156    3
Name: AnnotationEnumerated, Length: 1157, dtype: int64


In [244]:
#xiang_filtered_np = xiang_filtered["AnnotationEnumerated"].to_list()
ccad_filtered_np = ccad_filtered["AnnotationEnumerated"].to_list()

In [245]:
#ccad_filtered_np

In [246]:
#xiang_filtered["Annotation"].to_numpy()[5]

In [247]:
#ccad_filtered["kr_type_annotation"].to_numpy()[929]

In [248]:
#ccad_filtered_np[929]

In [249]:
#xiang_filtered_tensor = torch.tensor(xiang_filtered_np)
#xiang_filtered_tensor = [torch.tensor(x, dtype=torch.long) for x in xiang_filtered_np]

#xiang_filtered_tensor = torch.tensor(xiang_filtered_np, dtype=torch.long)
ccad_filtered_tensor = torch.tensor(ccad_filtered_np, dtype=torch.long)
#merged_tensor = torch.cat([xiang_filtered_tensor, ccad_filtered_tensor], dim=0)

In [250]:
#merged_embeddings = torch.cat([xiang_embeddings, ccad_embeddings], dim=0)

In [251]:
'''
from sklearn.model_selection import train_test_split

x_train_tensor_merged, x_test_tensor_merged, y_train_tensor_merged, y_test_tensor_merged = train_test_split(
    merged_embeddings,
    merged_tensor,
    test_size=0.2,
    random_state=1,
    stratify=merged_tensor
)

mu_merged, sigma_merged = x_train_tensor_merged.mean(0), x_train_tensor_merged.std(0) + 1e-9

x_train_tensor_merged = (x_train_tensor_merged - mu_merged) / sigma_merged
x_test_tensor_merged = (x_test_tensor_merged - mu_merged) / sigma_merged

print("x train len")
print(len(x_train_tensor_merged))
print("y train len")
print(len(y_train_tensor_merged))
'''

'\nfrom sklearn.model_selection import train_test_split\n\nx_train_tensor_merged, x_test_tensor_merged, y_train_tensor_merged, y_test_tensor_merged = train_test_split(\n    merged_embeddings,\n    merged_tensor,\n    test_size=0.2,\n    random_state=1,\n    stratify=merged_tensor\n)\n\nmu_merged, sigma_merged = x_train_tensor_merged.mean(0), x_train_tensor_merged.std(0) + 1e-9\n\nx_train_tensor_merged = (x_train_tensor_merged - mu_merged) / sigma_merged\nx_test_tensor_merged = (x_test_tensor_merged - mu_merged) / sigma_merged\n\nprint("x train len")\nprint(len(x_train_tensor_merged))\nprint("y train len")\nprint(len(y_train_tensor_merged))\n'

In [252]:
from sklearn.model_selection import train_test_split

x_train_tensor, x_test_tensor, y_train_tensor, y_test_tensor = train_test_split(
    ccad_embeddings,
    ccad_filtered_tensor,
    test_size = 0.3,
    random_state=1,
    stratify=ccad_filtered_tensor
)

mu, sigma = x_train_tensor.mean(0), x_train_tensor.std(0) + 1e-9
x_train_tensor = (x_train_tensor - mu) / sigma
x_test_tensor = (x_test_tensor - mu) / sigma

print("x train len")
print(len(x_train_tensor))
print("y train len")
print(len(y_train_tensor))

x train len
809
y train len
809


In [253]:
'''
            nn.BatchNorm1d(1024),
            nn.ReLU(),
            nn.Dropout(0.20),
            nn.Linear(1024, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Dropout(0.20),
            nn.Linear(512, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Dropout(0.20),
            nn.Linear(256,128),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Dropout(0.20),
'''

'\n            nn.BatchNorm1d(1024),\n            nn.ReLU(),\n            nn.Dropout(0.20),\n            nn.Linear(1024, 512),\n            nn.BatchNorm1d(512),\n            nn.ReLU(),\n            nn.Dropout(0.20),\n            nn.Linear(512, 256),\n            nn.BatchNorm1d(256),\n            nn.ReLU(),\n            nn.Dropout(0.20),\n            nn.Linear(256,128),\n            nn.BatchNorm1d(128),\n            nn.ReLU(),\n            nn.Dropout(0.20),\n'

In [254]:
'''
#add dropout_rate. ==
class kr_predict(nn.Module):
    def __init__(self):
        super(kr_predict, self).__init__()
        self.hidden = nn.Sequential(
            nn.Linear(xiang_embeddings[0].shape[1], 256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256,128),
            nn.ReLU(),
            nn.Dropout(0.3),
        )
        self.out = nn.Linear(128, 9)
    def forward(self, x):
        x = x.view(x.size(0), -1) # flatten so we're removing 
        x = self.hidden(x)
        x = self.out(x)
        return x      
'''


class kr_predict(nn.Module):
    def __init__(self):
        super(kr_predict, self).__init__()
        self.hidden = nn.Sequential(
            nn.Linear(1536, 512),
            nn.ReLU(),
            nn.Dropout(0.4),
            nn.Linear(512, 256),
            nn.ReLU(), 
            nn.Dropout(0.4),
            #nn.Linear(256, 128),
            #nn.BatchNorm1d(128),
            #nn.ReLU(),
            #nn.Dropout(0.3)
        )
        self.out = nn.Linear(256, 9)
    def forward(self, x):
        #x = x.view(x.size(0), -1) # flatten so we're removing - modified so its done w/ squeeze
        x = self.hidden(x)
        x = self.out(x)
        return x 

In [255]:
#xiang_embeddings[0].shape[1]

In [256]:
model = kr_predict()

In [257]:
#apple silicon

device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
model.to(device)

if torch.backends.mps.is_available():
    print("MPS is available! Using Apple Silicon GPU.")
else:
    print("MPS is not available. CPU Fallback.")

MPS is available! Using Apple Silicon GPU.


In [ ]:
from collections import Counter
class_counts = Counter(y_train_tensor.numpy())
print(annotation_enumerated)
print("Class distribution:", class_counts)

total_samples = sum(class_counts.values())  # Sum the actual counts, not keys
class_weights = []
num_classes = 9

class_weights = torch.tensor([
    total_samples / (num_classes * class_counts.get(i, 1)) for i in range(num_classes)
], dtype=torch.float32).to(device)

loss = nn.CrossEntropyLoss(weight=class_weights.to(device))
adam = optim.AdamW(model.parameters(), lr=0.001)
#scheduler = optim.lr_scheduler.CosineAnnealingLR(adam, T_max=50)
scheduler = optim.lr_scheduler.StepLR(adam, step_size=100, gamma=0.5)
#every 200 epochs, cut in half 0.5

{'A': 0, 'A1': 1, 'A2': 2, 'B': 3, 'B1': 4, 'B2': 5, 'C': 6, 'C1': 7, 'C2': 8}
Class distribution: Counter({3: 344, 4: 196, 0: 91, 1: 56, 6: 44, 2: 26, 7: 21, 5: 16, 8: 15})


In [259]:
x_train_tensor = x_train_tensor
y_train_tensor = (y_train_tensor).long()

In [260]:
print(y_train_tensor)

tensor([3, 3, 6, 6, 3, 4, 3, 3, 4, 6, 3, 3, 7, 3, 3, 4, 4, 4, 1, 4, 0, 4, 4, 1,
        3, 3, 1, 3, 3, 6, 3, 4, 3, 3, 3, 3, 3, 3, 3, 3, 6, 0, 0, 4, 3, 1, 4, 4,
        7, 3, 3, 1, 8, 1, 4, 4, 4, 3, 4, 4, 3, 3, 3, 3, 3, 3, 3, 3, 1, 6, 3, 3,
        4, 4, 1, 7, 0, 3, 4, 6, 7, 4, 3, 4, 3, 5, 3, 5, 4, 0, 3, 4, 3, 3, 3, 3,
        3, 0, 4, 4, 4, 1, 4, 0, 3, 2, 0, 3, 3, 3, 4, 0, 3, 3, 3, 3, 4, 4, 0, 3,
        4, 3, 3, 0, 4, 3, 0, 3, 3, 3, 3, 3, 4, 0, 3, 6, 3, 3, 4, 3, 3, 3, 3, 3,
        2, 3, 4, 4, 3, 4, 3, 7, 3, 4, 4, 4, 5, 4, 4, 3, 4, 4, 1, 4, 3, 5, 3, 3,
        3, 3, 6, 4, 4, 3, 1, 0, 2, 3, 3, 6, 4, 4, 4, 3, 3, 5, 4, 3, 6, 3, 4, 3,
        3, 3, 0, 3, 4, 4, 0, 3, 4, 3, 4, 4, 3, 3, 0, 3, 3, 0, 0, 4, 3, 3, 3, 3,
        4, 3, 1, 1, 4, 4, 1, 3, 3, 4, 3, 3, 4, 3, 6, 2, 3, 3, 5, 1, 4, 0, 0, 3,
        3, 4, 3, 3, 3, 8, 3, 3, 6, 3, 7, 3, 3, 6, 4, 3, 7, 3, 3, 3, 4, 2, 2, 3,
        3, 6, 3, 3, 3, 0, 4, 0, 4, 6, 1, 4, 3, 3, 4, 1, 3, 1, 3, 1, 3, 3, 0, 2,
        6, 4, 6, 7, 4, 3, 4, 0, 7, 3, 0,

In [ ]:

#use TensorDatset, DataLoader from torch utils

#worked well on gamma 0.5, epoch 550, batch 8

#batch_size = 32
batch_size = 16
train_dataset = TensorDataset(x_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, drop_last=True)
#dropping last bc we are using batchnorm, so it needs >1 batch size 

#1500 optimal
for epoch in range(550):
    model.train()
    epoch_loss = 0.0

    for seqs, anns in tqdm(train_loader, desc=f"Epoch {epoch+1}"):
        seqs = seqs.to(device)
        anns = anns.to(device)
        output = model(seqs)
        output_loss = loss(output, anns)
        adam.zero_grad()
        output_loss.backward()
        adam.step()
        epoch_loss += output_loss.item() * seqs.size(0) #batch size scaling
    scheduler.step()
    avg_loss = epoch_loss / len(train_dataset)
    print(f"Epoch {epoch+1}: Loss: {avg_loss:.4f}")


Epoch 1: 100%|█████████████████████████████████| 50/50 [00:00<00:00, 171.62it/s]


Epoch 1: Loss: 1.8359


Epoch 2: 100%|█████████████████████████████████| 50/50 [00:00<00:00, 333.36it/s]


Epoch 2: Loss: 1.2341


Epoch 3: 100%|█████████████████████████████████| 50/50 [00:00<00:00, 371.44it/s]


...

Epoch 549: 100%|███████████████████████████████| 50/50 [00:00<00:00, 296.09it/s]


Epoch 549: Loss: 0.0034


Epoch 550: 100%|███████████████████████████████| 50/50 [00:00<00:00, 295.45it/s]

Epoch 550: Loss: 0.0060


'\nimport matplotlib.pyplot as plt\nfrom IPython.display import clear_output\nimport time\n\n# Setup for ultra-fast live plotting\nplt.ioff()  # Turn off interactive mode\nepochs_list = []\nlosses_list = []\n\n# Training setup\nbatch_size = 8\ntrain_dataset = TensorDataset(x_train_tensor, y_train_tensor)\ntrain_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, drop_last=True)\n\nprint("Starting training with live loss visualization...")\nprint("Updates every single epoch")\n\nfor epoch in range(1000):\n    model.train()\n    epoch_loss = 0.0\n    \n    # loop (removed tqdm)\n    for seqs, anns in train_loader:\n        seqs = seqs.to(device)\n        anns = anns.to(device)\n        output = model(seqs)\n        output_loss = loss(output, anns)\n        adam.zero_grad()\n        output_loss.backward()\n        adam.step()\n        epoch_loss += output_loss.item() * seqs.size(0)\n    \n    # average loss\n    avg_loss = epoch_loss / len(train_dataset)\n    epochs_li

In [262]:
test_dataset = TensorDataset(x_test_tensor, y_test_tensor)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
#dropping last bc we are using batchnorm, so it needs >1 batch size 

all_predictions = []
all_targets = []
def accuracy():
    model.eval()
    correct = 0
    total = 0


    with torch.no_grad():
        for inputs, targets in test_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs) #model predictions [batch_size, 9]
            values, predicted = torch.max(outputs.data, 1)
            #values has highest score for each sample in batch
            #the predicted part has the classes w/ highest score for each sample
            total += targets.size(0) #add batch size
            correct += (predicted == targets).sum().item()

            #for classification report
            all_predictions.extend(predicted.cpu().numpy())
            all_targets.extend(targets.cpu().numpy())
    return 100 * correct/total

accuracy()

76.4367816091954

In [210]:

from sklearn.metrics import classification_report
import numpy as np


class_names = annotation_enumerated.keys()

print(classification_report(all_targets, all_predictions, target_names=class_names))

              precision    recall  f1-score   support

           A       0.72      0.72      0.72        36
          A1       0.62      0.65      0.64        23
          A2       0.80      0.73      0.76        11
           B       0.79      0.81      0.80       131
          B1       0.68      0.63      0.65        78
          B2       0.50      1.00      0.67         6
           C       0.73      0.65      0.69        17
          C1       0.44      0.44      0.44         9
          C2       0.80      0.67      0.73         6

    accuracy                           0.72       317
   macro avg       0.68      0.70      0.68       317
weighted avg       0.73      0.72      0.72       317

